# CML APIv2를 사용한 자동화된 앱 배포
응용 프로그램에 대한 관련 벡터로 Pinecone을 채운 후 이 노트북을 사용하면 이제 CML에서 앱을 배포할 준비가 됩니다.

#### 3.1 CML API client를 통해 사용 가능한 런타임 확인

이 코드는 Nvidia GPU 지원,  JupyterLab 편집기,  Python 3.10 버전 등의 검색 조건을 통해 사용 가능한 런타임 목록을 검색합니다. 
검색 결과의 runtime 중 하나를 선택하여 Application 배포에 활용합니다.

In [1]:
import os
import cmlapi
import random
import string
import json

client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
available_runtimes = client.list_runtimes(search_filter=json.dumps({
    "kernel": "Python 3.10",
    "edition": "Nvidia GPU",
    "editor": "JupyterLab",
    "full_version": "2024.05.1-b8"
}))
selected_runtime = max(available_runtimes.runtimes)


## Set available runtimes to the latest runtime in the environment (iterator is the number that begins with 0 and advances sequentially)
## The JOB_IMAGE_ML_RUNTIME variable stores the ML Runtime which will be used to launch the job
print(selected_runtime)
print(selected_runtime.image_identifier)
APP_IMAGE_ML_RUNTIME = selected_runtime.image_identifier

## Store the ML Runtime for any future jobs in an environment variable so we don't have to do this step again
os.environ['APP_IMAGE_ML_RUNTIME'] = APP_IMAGE_ML_RUNTIME

{'description': 'Python runtime with CUDA libraries provided by Cloudera',
 'edition': 'Nvidia GPU',
 'editor': 'JupyterLab',
 'full_version': '2024.05.1-b8',
 'image_identifier': 'docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-cuda:2024.05.1-b8',
 'kernel': 'Python 3.10',
 'register_user_id': 0,
 'status': 'ENABLED'}
docker.repository.cloudera.com/cloudera/cdsw/ml-runtime-jupyterlab-python3.10-cuda:2024.05.1-b8


#### 3.2 현재 작업 중인 프로젝트 정보 가져오기

여기서 우리는 환경 변수 "CDSW_PROJECT_ID"에서 현재 프로젝트를 가져오고 그 메타데이터를 인쇄합니다.

In [2]:
project = client.get_project(project_id=os.getenv("CDSW_PROJECT_ID"))
print(f"Current project: {project.name}")

Current project: Hands on Lab Workshop with LLM - hyeongyu_yeo1


#### 3.3 호스팅 LLM 애플리케이션에 대한 애플리케이션 생성 및 실행
이 코드는 호스팅할 LLM 어플리케이션에 대한 다음과 같은 조건 사항을 입력받아 요청합니다.
 - 입력 조건 : 이름("CML LLM Gradio Interface"), 설명, 호스팅할 프로젝트(현재 프로젝트), url에 표시할 sub domain (cml-llm-interface), 실행할 스크립트(3_launch_hosted_app/llm_prototype_app.py),  CPU,  메모리, 사용할 런타임 식별자 등

마지막으로 클라이언트를 사용하여 지정된 프로젝트 내에서 애플리케이션을 생성합니다.

In [7]:
application_request = cmlapi.CreateApplicationRequest(
     name = "CML LLM Gradio Interface",
     description = "Hosted interface for the CML LLM Gradio UI",
     project_id = project.id,
     subdomain = "cml-llm-interface",
     script = "3_launch_hosted_app/llm_prototype_app.py",
     cpu = 2,
     memory = 8,
     runtime_identifier = os.getenv('APP_IMAGE_ML_RUNTIME')
)


In [8]:
app = client.create_application(
     project_id = project.id,
     body = application_request
)

In [8]:
app = client.create_application(
     project_id = project.id,
     body = application_request
)